# 1. Đặt vấn đề


Trong lập trình C hay C++ chúng ta hay làm việc với con trỏ. Trong một chương trình lớn chúng ta phải quản lý một số 
lượng lớn con trỏ khác nhau, sau khi dùng xong chúng ta luôn phải giải phóng vùng nhớ con trỏ đó đang chỉ tới.

In [19]:
{
int* p = new int [10];
delete[] p; // giải phóng vùng nhớ  
}


Hoặc trong quá trình sử dụng ta có thể vô tình giải phóng con trỏ đó 2 lần. Hành động này dẫn đến lúc runtime bị **Undefined Behavior**.

In [20]:
// Các bạn mà xóa dấu commnet ở dưới rồi chạy là jupyter notebook error -> reset đấy nha =)))
{
int* p = new int [10];
    {
        delete[] p;
    }
    
//delete[] p;  // Undefined Behavior
}

>Sau khi chúng ta cấp phát bộ nhớ cho dùng cho chương trình thì ta cần phải xóa con trỏ đó đi để không gây lãng phí memory, leak memory. Với chương trình nhỏ việc quản lý các con trỏ thủ công tương đối dễ dàng, Tuy nhiên với các chương trình có mã nguồn lớn việc quản lý con trỏ thô này khá là khó khăn. Các anh em developer C thầm nghĩ đó là chuyện cơm bữa rồi, tuy nhiên với ngôn ngữ C++, những feature mới người ta có sáng tạo ra con trỏ thông minh - **Smart Poiter**. Thứ giúp ta quản lý con trỏ một cách tự động, ta không cần phải lo nghĩ đến việc phải giải phóng bộ nhớ mỗi khi dùng xong con trỏ. Vậy ta hãy đi tìm hiểu xem nó thực sự có **Smart** không thật nha. 

# 2. Cơ chế liên quan giải phóng bộ nhớ

Đây mình lấy ví dụ cho thấy nếu con trỏ không xóa thì việc gì sẽ xảy ra. 

In [21]:
#include <iostream>
class Memory {
public:
Memory() {
    std::cout << "Cấp phát vùng nhớ\n";
}

~Memory() {
    std::cout << "Thu hồi vùng nhớ\n";
    
}
};


{
    Memory* pM = new Memory();
}

std::cout << "---------------------\n";
{
   Memory m;
}


std::cout << "---------------------\n";
{
   Memory* pM = new Memory();
    delete pM;
    //pM->~Memory();
}


Cấp phát vùng nhớ
---------------------
Cấp phát vùng nhớ
Thu hồi vùng nhớ
---------------------
Cấp phát vùng nhớ
Thu hồi vùng nhớ


Kết quả run xong các bạn sẽ thấy thắc mắc tại sao thằng *pM không gọi được tới hàm hủy. Vì **Memory m** được khởi tạo tại stack nên khi out scope tự động gọi hàm Destructor. Còn **Memory \*pM** giá trị của con trỏ *pM được khởi tạo tại **Stack**, còn dữ liệu được trỏ tới đối tượng **Memory** được khởi tạo tại **Heap**. Với dự liệu được khởi tạo trong **Heap** khác với dữ liệu khởi tạo tại **Stack** là chúng ta phải quản lý chúng một cách thủ công. Nên là khi con trỏ **\*pM** out scope thì tuy giá trị con trỏ được phóng nhưng dữ liệu con trỏ tới tại **Heap** chưa được giải phóng --> leak memory.


[Tham khảo thêm.](https://stackoverflow.com/questions/56588367/why-is-the-destructor-not-called-when-im-declaring-a-pointer-to-an-object)


# 3. Bí mật đằng sau Smart Poiter

Ta xem qua ví dụ code về **Smart pointer** nhé:


``` cpp
//Các bạn code trình biên dịch nào đó họăc c++ compiler online run cũng được. Vì code dưới không thể chạy được trên xeus-cling. :((((
#include <iostream>

class Memory {
public:
    Memory() {
        std::cout << "Memory Constructor\n";
    }

    ~Memory() {
        std::cout << "Memory Destructor\n";
    }
};

template <typename T>
class SmartPointer {
public:
    SmartPointer() {
        std::cout << "SmartPointer Constructor\n";
        t = new T();
    }

    ~SmartPointer() {
        std::cout << "SmartPointer Destructor\n";
        delete t;
    }

    SmartPointer(const SmartPointer& other) {
        std::cout << "SmartPointer Copy Constructor\n";
        t = new T(*other.t); // Sao chép đối tượng từ pointer
    }

    // Toán tử gán - ngăn gán sai cách
    SmartPointer& operator=(const SmartPointer& other) {
        if (this == &other) return *this; // Tránh tự gán

        delete t; // Giải phóng bộ nhớ hiện tại
        t = new T(*other.t); // Sao chép đối tượng
        return *this;
    }


    
private:
    T* t;
};
int main() {
 {
    SmartPoiter<Memory> pSmart; 
}
    return 0;
}
```

Các bạn đọc xong code ngẫm nghĩ chút rồi đoán xem ý tưởng của **Smart Poiter** là gì. Ý tưởng của nó thực ra là dựa trên ý tưởng là 1 class sẽ wrap(SmartPointer) là lại dữ liệu được cấp phát tại heap mà ta muốn quản lý. KỸ hơn là giá trị khởi tạo pSmart được khởi tạo trong **Stack** nên khi out scpoe thì pSmart này sẽ tự động gọi đến hàm Destructor của nó, rồi nó sẽ gọi tới hàm hàm delete. Từ đó giải phóng giữ liệu trên heap của dữ liệu **T** mà class SmartPointer quản lý.


> Riêng có 2 hàm Copy Constructor và Move Constructor là để cho mục đích tạo ra các loại **Smart Poiter** mà mình sẽ đề cập bên dưới.

# 3. Phân loại các loại **Smart Pointer**

Có các loại smart poiter chính:
- unique_ptr
- shared_ptr
- weak_ptr
- auto_ptr

> Trong nội dung bài này mình sẽ không nói đến auto_ptr vì người ta thường chỉ sử dụng unique_ptr, shared_ptr, weak_ptr.

## 3.1 Unique_ptr

Con trỏ **unique_ptr** là con trỏ smart sở hữu, quản lý đối tượng khác thông qua con trỏ và xử lý giải phóng bộ nhớ nếu unique_ptr out scope hoặc chuyển dữ liệu sang con trỏ khác quản lý.

Đối tượng được con trỏ unique_ptr quản lý giải phóng dựa trên hàm deleter:
- Khi con trỏ uinque_ptr thoát khỏi vùng tồn tại(out scope)
- Khi con trỏ uinque_ptr chuyển quyền sở hữu cho con trỏ unique_ptr khác, gọi hàm **reset()**.

> Cơ chế hoạt hoạt động đằng sau unique_ptr là: **Unique_ptr** = **Smart Pointer** + **Move assginment, contructor** - **Copy assginment, contructor**

```cpp
template <typename T>
class UniquePointer {
public:
    UniquePointer() {
        std::cout << "UniquePointer Constructor\n";
        t = new T();
    }

    ~UniquePointer() {
        std::cout << "UniquePointer Destructor\n";
        delete t;
    }

    UniquePointer(UniquePointer&& ohter) {
        t = other.t;
        other.t = nullptr;
    }

    UniquePointer& opterator=(UniquePointer&& ohter) {
        if (*this != other) {
            delete t;

            t = other.t;
            other.t = nullptr;
        }
        return *this;

    }

    UniquePointer(const UniquePointer& other) = delete;   //copy contructor
    UniquePointer& operator=(const UniquePointer& other) = delete; // copy assignment


private:
    T* t;
};


```

## 3.2 Shared_ptr

Con trỏ **shared_ptr** là con trỏ giữ quyền sở hữu chung đến đối tượng thông qua con trỏ. Nhiều con trỏ **share_ptr** cùng trỏ đến một đối tượng duy nhất. Đối tượng con trỏ shard_ptr bị hủy(destroyed) và vùng nhớ của nó bị giải phóng(deallocated) khi:
- Con trỏ shared_ptr cuối cùng trỏ đến đối tượng bị hủy(out scope).
- Con trỏ shared_ptr cuối cùng trỏ đến đối tương chuuyển quyền quản lý cho loại con trỏ khác(**unique_ptr**).

> Với con trỏ **Shared_ptr** = **Smart Pointer** + **Move assginment, contructor** + **Copy assginment, contructor**. Shared_ptr còn hoạt động dựa trên biến use_count(số tham chiếu đến đối tượng)

Mình sẽ chú ý vào các ý quan trọng trong implement của **Shared_ptr**. Con trỏ **shared_ptr** gồm:
- Con trỏ thô(raw pointer) trỏ đến đối tượng ta quản lý.
- Con trỏ *conntrol block*. Cấu trúc của con trỏ *control block*:
  + Con trỏ đến đối tượng ta quản lý hoặc con trỏ trỏ đến đối tượng ta quản lý. 
  + Biến use_count - số các shared_ptr đang cùng quản lý đối tượng.
  + Biến weak_count- số các weak_ptr đang trỏ đến đối tượng ta quản lý.
  + Hàm xóa - the deleter (type-erased)s
  + Hàm cấp phát - the allocator (type-erased)

> Lưu ý: Nếu use_count = 0 thì đối tượng quản lý của ta sẽ bị hủy(deleter), nếu user_count = 0 và weak_count = 0 thì con trỏ **Control block** sẽ bị hủy

Về triển khai của **Shared_ptr** khá là khó. Bên dưới là code tham khảo bản implement cơ bản của **Shared_ptr**. Nếu các bạn muốn hiểu rõ hơn thì các bạn có thể đọc implement trong thư viện =)))). 

[Link bài viết implement code tham khảo bên dưới](https://medium.com/analytics-vidhya/c-shared-ptr-and-how-to-write-your-own-d0d385c118ad)



```cpp
typedef unsigned int uint;

template<class T>
class my_shared_ptr
{
private:
	T * ptr = nullptr;
	uint * refCount = nullptr;

public:
	my_shared_ptr() : ptr(nullptr), refCount(new uint(0)) // default constructor
	{
	}

	my_shared_ptr(T * ptr) : ptr(ptr), refCount(new uint(1)) // constructor
	{
	}

	/*** Copy Semantics ***/
	my_shared_ptr(const my_shared_ptr & obj) // copy constructor
	{
		this->ptr = obj.ptr; // share the underlying pointer
		this->refCount = obj.refCount;
		if (nullptr != obj.ptr)
		{
			(*this->refCount)++; // if the pointer is not null, increment the refCount
		}
	}

	my_shared_ptr& operator=(const my_shared_ptr & obj) // copy assignment
	{
		__cleanup__(); // cleanup any existing data
		
		// Assign incoming object's data to this object
		this->ptr = obj.ptr; // share the underlying pointer
		this->refCount = obj.refCount;
		if (nullptr != obj.ptr)
		{
			(*this->refCount)++; // if the pointer is not null, increment the refCount
		}
	}

	/*** Move Semantics ***/
	my_shared_ptr(my_shared_ptr && dyingObj) // move constructor
	{
		this->ptr = dyingObj.ptr; // share the underlying pointer
		this->refCount = dyingObj.refCount;

		dyingObj.ptr = dyingObj.refCount = nullptr; // clean the dying object
	}

	my_shared_ptr& operator=(my_shared_ptr && dyingObj) // move assignment
	{
		__cleanup__(); // cleanup any existing data
		
		this->ptr = dyingObj.ptr; // share the underlying pointer
		this->refCount = dyingObj.refCount;

		dyingObj.ptr = dyingObj.refCount = nullptr; // clean the dying object
	}

	uint get_count() const
	{
		return *refCount; // *this->refCount
	}

	T* get() const
	{
		return this->ptr;
	}

	T* operator->() const
	{
		return this->ptr;
	}

	T& operator*() const
	{
		return this->ptr;
	}

	~my_shared_ptr() // destructor
	{
		__cleanup__();
	}

private:
	void __cleanup__()
	{
		(*refCount)--;
		if (*refCount == 0)
		{
			if (nullptr != ptr)
				delete ptr;
			delete refCount;
		}
	}
};

```

#### To do chèn thêm ảnh minh họa về sử dụng **shared_ptr** trong dự án thực tế.

## 3.3 weak_ptr

Con trỏ **weak_ptr** là con trỏ giữ một tham chiếu lỏng lẻo ( non-owning weak reference) đến đối tượng ta quản lý trong đó đối tượng quản lý đó cũng phải đang được quản lý bới con trỏ **shared_ptr**.  

- Con trỏ weak_ptr là một mô hình yếu. Nó thực hiện truy cập đối tượng quản lý khi mà đối tượng quản lý đó còn tồn tại. Nó không thể tự giữ tham chiếu đến đối tượng mà phải được thông qua việc được conver thành **shared_ptr** temporary cho quyền sở hữu tạm thời, Và **shared_ptr** temporary sẽ kéo dài **life time** của object quản lý khi mà **shared_ptr** thực sự của object đã bị hủy hết. (Hay một cách hiểu khác là **weak_ptr** là một **shared_ptr observer**.)
- Một công dụng khác của **weak_ptr** là để break ra cycles có thể có do **shared_ptr** gây ra.

```cpp
#include <iostream>
#include <memory>
#include <variant>
 
class Node
{
    char id;
    std::variant<std::weak_ptr<Node>, std::shared_ptr<Node>> ptr;
public:
    Node(char id) : id{id} {}
    ~Node() { std::cout << "  '" << id << "' reclaimed\n"; }
    /*...*/
    void assign(std::weak_ptr<Node> p) { ptr = p; }
    void assign(std::shared_ptr<Node> p) { ptr = p; }
};
 
enum class shared { all, some };
 
void test_cyclic_graph(const shared x)
{
    auto A = std::make_shared<Node>('A');
    auto B = std::make_shared<Node>('B');
    auto C = std::make_shared<Node>('C');
 
    A->assign(B);
    B->assign(C);
 
    if (shared::all == x)
    {
        C->assign(A);
        std::cout << "All links are shared pointers";
    }
    else
    {
        C->assign(std::weak_ptr<Node>(A));
        std::cout << "One link is a weak_ptr";
    }
    /*...*/
    std::cout << "\nLeaving...\n";
}
 
int main()
{
    test_cyclic_graph(shared::some);
    test_cyclic_graph(shared::all); // produces a memory leak
}


```

# 4. Ưu và nhược điểm của **Smart Poiter**
Theo tài liệu tham khảo của [Google Style C++](http://google-styleguide.googlecode.com/svn/trunk/cppguide.xml#Smart_Pointers)

Ưu điểm:
- Tự động quản lý bộ nhớ, tránh gây leak memory, chuyền quyền sở hữu vùng nhớ một cách rõ ràng


Nhược điểm:
- Thiết kế con trỏ mà nó chỉ có sở hữu một đối tượng duy nhất. Trong đó lưu ý tới con auto_ptr có thể gây ra lỗi khó phát hiện.

Tuy nhiên theo kinh nghiệm làm việc với **Smart Pointer**, mình thấy dùng **unique_ptr** với trường hợp class muốn chính nó là người sở hữu 1 vùng dữ liệu nào đấy. **shard_ptr** là dữ liệu đó được dùng chung giữa các class. Còn **weak_ptr** là để kiểm tra xem dữ liệu đó còn tồn tại không thì thực hiện một số hành động nào đó.

> Câu hỏi phỏng vấn khi nào dùng **Smart Poiter** và khi nào dùng con trỏ thông thường (raw poiter):

>Trả lời: Dùng **Smart Pointer** trong trường hợp muốn thực hiện một số thao tác như sở hữu riêng hoặc chia sẽ quyền sở hữu, truyền quyền sở hữu, theo dõi **lifetime** của đối tượng. Dùng raw pointer khi mà pointer đó chỉ nhiệm vụ trỏ đến một đối tượng duy nhất.

>Câu hỏi phỏng vấn liệu use_count của shard_ptr có thể tăng vô hạn không ?

# 5.Tài liệu tham khảo

1. [unique_ptr cppreference](https://en.cppreference.com/w/cpp/memory/unique_ptr)

2. [shared_ptr cppreference](https://en.cppreference.com/w/cpp/memory/shared_ptr)

3. [weak_ptr cppreference](https://en.cppreference.com/w/cpp/memory/weak_ptr)